# Validate simplified linear forecast model

## Setup

In [2]:
import sys
toolpath = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/src'
sys.path.append(toolpath)

from wtfc_utils import etl_utils as etl

from sqlalchemy import create_engine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

## Load Data

In [3]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/'
db_dir = server_dir + 'db/'
db_name = 'weightforecaster'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

engine = create_engine('sqlite:///'+db_file_name)

with engine.connect() as conn, conn.begin():
    db_df = pd.read_sql_table('fitness', conn, index_col='date', parse_dates=['date'])

In [4]:
db_df.tail(8)

,weight,calories,steps,weight_imputed,w_7day_avg,c_7day_avg,s_7day_avg,w_7day_avg_last_week,c_7day_avg_last_week,s_7day_avg_last_week,w_7day_avg_weekly_diff
date,,,,,,,,,,,
2021-07-01,158.8,1957.0,11489.0,0.0,158.342857,2110.285714,13586.571429,158.885714,2212.857143,16943.285714,-0.542857
2021-07-02,156.5,3267.0,20434.0,0.0,157.857143,2263.714286,14845.000000,159.400000,2250.714286,16607.714286,-1.542857
2021-07-03,158.1,2559.0,23734.0,0.0,157.500000,2288.000000,16686.428571,159.528571,2266.714286,16490.857143,-2.028571
2021-07-04,159.3,2289.0,16903.0,0.0,157.842857,2313.142857,17260.714286,159.385714,2221.714286,16043.285714,-1.542857
2021-07-05,157.5,2108.0,16527.0,0.0,158.071429,2280.857143,16800.857143,158.628571,2074.285714,16350.714286,-0.557143
2021-07-06,155.7,1881.0,17462.0,0.0,157.800000,2288.285714,17242.142857,158.214286,2106.714286,15129.571429,-0.414286
2021-07-07,157.7,1719.0,12459.0,0.0,157.657143,2254.285714,17001.142857,158.171429,2115.000000,14462.571429,-0.514286
2021-07-08,155.4,1656.0,11330.0,0.0,157.171429,2211.285714,16978.428571,158.342857,2110.285714,13586.571429,-1.171429


## Numerical solution

In [65]:
model_coefs = [0.9842664081035283, # c_w
               0.001965638199353011, # c_c
               -4.621900527451458e-05, # c_s
               -1.2110620297640367] # c_0
[c_w, c_c, c_s, c_0] = model_coefs

alpha_s = -c_s/c_c
alpha_0 = -c_0/c_c
alpha_w = (1-c_w)/c_c

def wss(C, S):
    return (C - alpha_s*S - alpha_0)/alpha_w

def css(W, S):
    return (alpha_s*S + alpha_0 + alpha_w*W)

In [69]:
num_steps = 300
temp_df = pd.DataFrame({'step':list(range(num_steps))})

c = 2132
s = 10000
w0 = 180
gamma = -np.log(c_w)

w_prev = w0
temp_df['w'] = w_prev
for t in list(temp_df.step)[1:]:    
    w_prev = c_0 + c_w*w_prev + c_c*c + c_s*s
    temp_df.loc[temp_df.step==t, 'w'] = w_prev
    
winf = wss(c,s)
w_soln = []
w_lin = []
for t in temp_df.step:
    w1 = (w0 - winf)*np.exp(-gamma*t) + winf
    wlin = w0 + (winf - w0)*gamma*t
    w_soln.append(w1)
    w_lin.append(wlin)
temp_df['w_soln'] = w_soln
temp_df['w_lin'] = w_lin

In [70]:
ch1=alt.Chart(temp_df).mark_point(size=10, opacity=1, color='#222', fill='#222').encode(
    x=alt.X('step', scale=alt.Scale(domain=(0,300)), title='weeks'),  
    y=alt.Y('w', title='weight',
        scale=alt.Scale(domain=(155, 180))
    )
).properties(
    width=900,
    height=450
).interactive()

ch2=alt.Chart(temp_df).mark_line(color='red').encode(
    x=alt.X('step', scale=alt.Scale(domain=(0,300)), title='weeks'),  
    y=alt.Y('w_soln', title='weight',
        scale=alt.Scale(domain=(155, 180))
    )
).properties(
    width=900,
    height=450
).interactive()

ch3=alt.Chart(temp_df).mark_line(color='blue').encode(
    x=alt.X('step', scale=alt.Scale(domain=(0,300)), title='weeks'),  
    y=alt.Y('w_lin', title='weight',
        scale=alt.Scale(domain=(155, 180))
    )
).properties(
    width=900,
    height=450
).interactive()

(ch1 + ch2 + ch3).configure_axis(
    labelFontSize=16,
    titleFontSize=16
)

alt.LayerChart(...)